Import Library

In [ ]:
# Library utama untuk manipulasi data
import pandas as pd
import numpy as np

# Library untuk model regresi
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor # Import RandomForestRegressor

# Library untuk evaluasi model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Untuk menyimpan model
import joblib

Load Dataset

In [ ]:
from google.colab import files

# Membuka dialog upload file dari laptop
uploaded = files.upload()


Saving Coffee Shop Sales.xlsx to Coffee Shop Sales (1).xlsx


In [ ]:
# Load dataset dari file Excel
# Pastikan file sudah di-upload ke Colab
df = pd.read_excel("Coffee Shop Sales.xlsx")

# Lihat 5 data teratas
df.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
0,1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg
1,2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
2,3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
3,4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm
4,5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg


Seleksi Kolom yang Digunakan

In [ ]:
# Ambil hanya kolom yang relevan dengan tujuan sistem
df = df[
    [
        'transaction_id',       # identitas transaksi
        'transaction_date',     # tanggal transaksi
        'transaction_time',     # waktu transaksi
        'transaction_qty',      # TARGET (jumlah terjual)
        'product_id',           # identitas produk
        'product_category',     # kategori produk
        'product_type',         # tipe produk
        'unit_price'            # harga satuan
    ]
]

df.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,product_id,product_category,product_type,unit_price
0,1,2023-01-01,07:06:11,2,32,Coffee,Gourmet brewed coffee,3.0
1,2,2023-01-01,07:08:56,2,57,Tea,Brewed Chai tea,3.1
2,3,2023-01-01,07:14:04,2,59,Drinking Chocolate,Hot chocolate,4.5
3,4,2023-01-01,07:20:24,1,22,Coffee,Drip coffee,2.0
4,5,2023-01-01,07:22:41,2,57,Tea,Brewed Chai tea,3.1


Pre-processing

In [ ]:
# Cek jumlah missing value sebelum dibersihkan
print("Missing value sebelum:\n", df.isnull().sum())

# Cek jumlah data duplikat sebelum dibersihkan
print("\nJumlah duplikat sebelum:", df.duplicated().sum())



Missing value sebelum:
 transaction_id      0
transaction_time    0
transaction_qty     0
product_id          0
product_category    0
product_type        0
unit_price          0
dtype: int64

Jumlah duplikat sebelum: 0


In [ ]:
# Hapus SEMUA baris yang punya missing value
df = df.dropna()

# Hapus SEMUA data duplikat
df = df.drop_duplicates()

In [ ]:
# Cek ulang setelah preprocessing
print("\nMissing value sesudah:\n", df.isnull().sum())
print("\nJumlah duplikat sesudah:", df.duplicated().sum())

# Lihat ukuran data akhir
print("\nJumlah data akhir:", df.shape)


Missing value sesudah:
 transaction_id      0
transaction_time    0
transaction_qty     0
product_id          0
product_category    0
product_type        0
unit_price          0
dtype: int64

Jumlah duplikat sesudah: 0

Jumlah data akhir: (149116, 7)


Pisahkan Fitur (X) dan Target (y)

In [ ]:
# ================================
# FEATURE ENGINEERING WAKTU (FINAL)
# ================================

# Pastikan tanggal bertipe datetime
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# Pastikan transaction_time juga sudah diubah ke datetime agar bisa diekstraksi komponen waktunya
# Jika belum, lakukan konversi dari string ke datetime
# Asumsi di sini transaction_time sudah datetime dari proses sebelumnya atau data original
# Jika original adalah string 'HH:MM:SS', maka perlu diubah dulu:
# df['transaction_time'] = pd.to_datetime(df['transaction_time'], format='%H:%M:%S').dt.time

# Gabungkan tanggal + jam
df['transaction_datetime'] = pd.to_datetime(
    df['transaction_date'].astype(str) + ' ' + df['transaction_time'].dt.strftime('%H:%M:%S')
)

# Ekstraksi fitur waktu
df['hour'] = df['transaction_datetime'].dt.hour
df['day_of_week'] = df['transaction_datetime'].dt.dayofweek
df['month'] = df['transaction_datetime'].dt.month
df['weekend'] = (df['day_of_week'] >= 5).astype(int)

df[['transaction_datetime','hour','day_of_week','weekend']].head()

,transaction_datetime,hour,day_of_week,weekend
0,2023-01-01 07:06:11,7,6,1
1,2023-01-01 07:08:56,7,6,1
2,2023-01-01 07:14:04,7,6,1
3,2023-01-01 07:20:24,7,6,1
4,2023-01-01 07:22:41,7,6,1


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

# ================================
# ENCODING PRODUK
# ================================

# Encode product_type (mewakili jenis produk)
le_product = LabelEncoder()
df['product_type_encoded'] = le_product.fit_transform(df['product_type'])

# ================================
# FITUR (X) DAN TARGET (y)
# ================================

# Fitur input ke model
X = df[
    [
        'hour',                 # jam transaksi
        'day_of_week',          # hari (0=Senin)
        'month',                # bulan
        'weekend',              # weekend / weekday
        'product_type_encoded', # produk (hasil encoding)
        'unit_price'            # harga
    ]
]

# Target asli (disimpan untuk evaluasi)
y = df['transaction_qty']

# Target yang DIPAKAI model (log transform)
y_log = np.log1p(df['transaction_qty'])

# ================================
# SPLIT DATA TRAIN & TEST
# ================================

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_log,        # ✅ PAKAI y_log, BUKAN y
    test_size=0.2,
    random_state=42
)

print("Data training:", X_train.shape)
print("Data testing :", X_test.shape)


Data training: (119292, 6)
Data testing : (29824, 6)


In [ ]:
# Training model Linear Regression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Prediksi
y_pred_lr = lr_model.predict(X_test)

In [ ]:
# Training model Random Forest sebagai pembanding

rf_model = RandomForestRegressor(
    n_estimators=100,
    random_state=42
)

rf_model.fit(X_train, y_train)

# Prediksi
y_pred_rf = rf_model.predict(X_test)

In [ ]:
# Evaluasi Linear Regression

mae_lr = mean_absolute_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

# Evaluasi Random Forest
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

# Tampilkan hasil
results = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest'],
    'MAE': [mae_lr, mae_rf],
    'RMSE': [rmse_lr, rmse_rf],
    'R2 Score': [r2_lr, r2_rf]
})

results

,Model,MAE,RMSE,R2 Score
0,Linear Regression,0.198671,0.207797,0.03444
1,Random Forest,0.173994,0.213181,-0.01624


In [ ]:
# Simpan model terbaik berdasarkan R2 Score

best_model = rf_model if r2_rf > r2_lr else lr_model

joblib.dump(best_model, "model_prediksi_penjualan.pkl")
joblib.dump(le_product, "label_encoder.pkl")

print("Model dan encoder berhasil disimpan")

Model dan encoder berhasil disimpan


In [ ]:
model = joblib.load("model_prediksi_penjualan.pkl")
encoder = joblib.load("label_encoder.pkl")

In [ ]:
from google.colab import files
files.download("model_prediksi_penjualan.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("label_encoder.pkl")


In [ ]:
# ================================
# DATASET FINAL (FITUR TERPILIH)
# ================================

df_final = df[
    [
        'hour',
        'day_of_week',
        'month',
        'weekend',
        'product_type_encoded',
        'unit_price',
        'transaction_qty'
    ]
]

df_final.head()


,hour,day_of_week,month,weekend,product_type_encoded,unit_price,transaction_qty
0,7,6,1,1,13,3.0,2
1,7,6,1,1,4,3.1,2
2,7,6,1,1,17,4.5,2
3,7,6,1,1,10,2.0,1
4,7,6,1,1,4,3.1,2


In [ ]:
df_final.to_csv("dataset_preprocessing_final.csv", index=False)


In [ ]:
from google.colab import files

files.download("dataset_preprocessing_final.csv")
# atau
# files.download("dataset_preprocessing_final.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>